In [1]:
import numpy as np
import cv2
import os
from tqdm import tqdm
from time import time

from keras.preprocessing.image import img_to_array
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse



Using TensorFlow backend.


In [2]:
# import the necessary packages

class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        # store the target image width, height, and interpolation
        # method used when resizing
        self.width = width
        self.height = height
        self.inter = inter

    def preprocess(self, image):
        # resize the image to a fixed size, ignoring the aspect
        # ratio
        return cv2.resize(image, (self.width, self.height),interpolation=self.inter)

In [3]:
# import the necessary packages

class ImageToArrayPreprocessor:
    def __init__(self, dataFormat=None):
        # store the image data format
        self.dataFormat = dataFormat

    def preprocess(self, image):
        # apply the Keras utility function that correctly rearranges
        # the dimensions of the image
#         return img_to_array(image, data_format=self.dataFormat)
        return img_to_array(image)

In [4]:
class SimpleDatasetLoader:
#     def __init__(self, preprocessors=None):
    def __init__(self, width, height):
        from tqdm import tqdm
        self.width = width
        self.height = height



    def load(self, imagePaths, verbose=-1):
        # initialize the list of features and labels
        data = []
        labels = []

        for img in tqdm(os.listdir(imagePaths)):
            label = img
            path = os.path.join(imagePaths,img)
            for filename in os.listdir(path):
                image = cv2.imread(os.path.join(path, filename))
                sp = SimplePreprocessor(self.width, self.height)
                image = sp.preprocess(image)
#                 print ("sp completed")
                iap = ImageToArrayPreprocessor()
                image = iap.preprocess(image)

                data.append(image)
                labels.append(label)

        return (np.array(data), np.array(labels))


In [5]:
print("[INFO] loading images...")
imagePaths = 'G:/pyimagesearch/SB_Code/SB_Code/datasets/animals/'

sdl = SimpleDatasetLoader(width = 32, height = 32)
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0
print ('completed')

[INFO] loading images...


100%|████████████████████████████████████████████████████████████| 3/3 [00:13<00:00,  4.26s/it]


completed


In [6]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)
# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [7]:
trainX.shape

(2250, 32, 32, 3)

In [8]:
trainY.shape

(2250, 3)

In [9]:
32 * 32 * 3

3072

In [10]:
# IMG_SIZE = 80

# epochs = 5
# step_size = 8
# IMG_SIZE_ALEXNET = 227
# validating_size = 40
# nodes_fc1 = 4096
# nodes_fc2 = 4096
# output_classes = 4


In [11]:
#Resetting graph
tf.reset_default_graph()

#Defining Placeholders
# x = tf.placeholder(tf.float32,shape=[None,227,227,3])
x = tf.placeholder(tf.float32,shape=[None,32,32,3])
y_true = tf.placeholder(tf.float32,shape=[None,3])

In [44]:
##CONVOLUTION LAYER 1
#Weights for layer 1
w_1 = tf.Variable(tf.truncated_normal([11,11,3,32], stddev=0.01))
#Bias for layer 1
b_1 = tf.Variable(tf.constant(0.0, shape=[[11,11,3,32][3]]))
#Applying convolution
c_1 = tf.nn.conv2d(x, w_1,strides=[1, 4, 4, 1], padding='VALID')
#Adding bias
c_1 = c_1 + b_1
#Applying RELU

c_2 = tf.nn.batch_normalization(c_1, .2, .02, 1, 1, .00001111)
c_1 = tf.nn.relu(c_1)
print(c_1)

Tensor("Relu_4:0", shape=(?, 6, 6, 32), dtype=float32)


In [45]:
##POOLING LAYER1
p_1 = tf.nn.max_pool(c_1, ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1], padding='VALID')
print(p_1)

Tensor("MaxPool_2:0", shape=(?, 2, 2, 32), dtype=float32)


In [46]:
##CONVOLUTION LAYER 2
#Weights for layer 2
w_2 = tf.Variable(tf.truncated_normal([5,5,32,64], stddev=0.01))
#Bias for layer 2
b_2 = tf.Variable(tf.constant(1.0, shape=[[5,5,32,64][3]]))
#Applying convolution
c_2 = tf.nn.conv2d(p_1, w_2,strides=[1, 1, 1, 1], padding='SAME')
#Adding bias
c_2 = c_2 + b_2
#Applying RELU
c_2 = tf.nn.relu(c_2)

print(c_2)

Tensor("Relu_5:0", shape=(?, 2, 2, 64), dtype=float32)


In [47]:
##POOLING LAYER2
p_2 = tf.nn.max_pool(c_2, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='VALID')
print(p_2)

Tensor("MaxPool_3:0", shape=(?, 1, 1, 64), dtype=float32)


In [48]:
#     layer_shape = layer.get_shape()
#     num_features = layer_shape[1:4].num_elements()
#     layer = tf.reshape(layer, [-1, num_features])

In [49]:
# flattened = tf.reshape(p_3,[-1,6*6*256])

num_features  = 3
layer_shape = p_2.get_shape()
num_features = layer_shape[1:4].num_elements()
flattened = tf.reshape(p_2, [-1, num_features])
# print(flattened)

In [50]:
input_size = int( flattened.get_shape()[1] )
#Weights for FC Layer 1
w1_fc = tf.Variable(tf.truncated_normal([input_size, 30], stddev=0.01))
#Bias for FC Layer 1
b1_fc = tf.Variable( tf.constant(1.0, shape=[30] ) )
#Summing Matrix calculations and bias
s_fc1 = tf.matmul(flattened, w1_fc) + b1_fc
#Applying RELU
s_fc1 = tf.nn.relu(s_fc1)

In [51]:
# #Dropout Layer 1
hold_prob1 = tf.placeholder(tf.float32)
s_fc1 = tf.nn.dropout(s_fc1,keep_prob=hold_prob1)
print(s_fc1)

Tensor("dropout_2/mul:0", shape=(?, 30), dtype=float32)


In [52]:
##Fully Connected Layer 2
#Weights for FC Layer 2
w2_fc = tf.Variable(tf.truncated_normal([30, 50], stddev=0.01))
#Bias for FC Layer 2
b2_fc = tf.Variable( tf.constant(1.0, shape=[50] ) )
#Summing Matrix calculations and bias
s_fc2 = tf.matmul(s_fc1, w2_fc) + b2_fc
#Applying RELU
s_fc2 = tf.nn.relu(s_fc2)
print(s_fc2)

Tensor("Relu_7:0", shape=(?, 50), dtype=float32)


In [53]:
#Dropout Layer 2
hold_prob2 = tf.placeholder(tf.float32)
s_fc2 = tf.nn.dropout(s_fc2,keep_prob=hold_prob1)

In [54]:
##Fully Connected Layer 3
#Weights for FC Layer 3
w3_fc = tf.Variable(tf.truncated_normal([50,3], stddev=0.01))
#Bias for FC Layer 3b3_fc = tf.Variable( tf.constant(1.0, shape=[output_classes] ) )
b3_fc = tf.Variable( tf.constant(1.0, shape=[3] ) )
#Summing Matrix calculations and bias
y_pred = tf.matmul(s_fc2, w3_fc) + b3_fc
#Applying RELU
print(y_pred)

Tensor("add_10:0", shape=(?, 3), dtype=float32)


In [55]:
#Defining loss function

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred))

#Defining objective
train = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(cross_entropy)

#Defining Accuracy
matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
acc = tf.reduce_mean(tf.cast(matches,tf.float32))

#Initializing weights
init = tf.global_variables_initializer()

In [56]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())

#     for i in range(30):
# #         batch = mnist.train.next_batch(50)
#         idx = np.random.choice(np.arange(len(trainX)), 100, replace=False)
#         x_sample = trainX[idx]
#         print (x_sample[1])
#         y_sample = trainY[idx]
#         print (y_sample[1])

# #         if i % 100 == 0:
# #             train_accuracy = sess.run(acc, feed_dict={x: x_sample, 
# #                                                            y_true: y_sample,
# #                                                            hold_prob1:0.5,hold_prob2:0.5})
# #             print ("step {}, training accuracy {}".format(i, train_accuracy))

#         sess.run(train, feed_dict={x: x_sample, y_true: y_sample,
#                                         hold_prob1:0.5,hold_prob2:0.5})
#         print ("step {}, training accuracy {}".format(i, train_accuracy))

# #     X = mnist.test.images.reshape(10, 1000, 784)
# #     Y = mnist.test.labels.reshape(10, 1000, 10)
#     test_accuracy = np.mean([sess.run(acc, 
#                              feed_dict={x:testX, y_true:testY,hold_prob1:1.0,hold_prob2:1.0})
#                              for i in range(10)])

# print ("test accuracy: {}".format(test_accuracy))



In [57]:
trainX.shape

(2250, 32, 32, 3)

In [58]:
trainY.shape

(2250, 3)

In [59]:
# concate_train = np.concatenate((trainX,trainY),axis=0)

In [60]:
# sess.run(tf.initialize_all_variables())

batch_size = 50

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for n in range( 300 ):
        print(n)
        batch_x = trainX[ n*batch_size : (n+1)*batch_size ]
        batch_y = trainY[ n*batch_size : (n+1)*batch_size ]
        train_accuracy = sess.run(acc, feed_dict={x: batch_x, 
                                                  y_true: batch_y,
                                                  hold_prob1:0.7,hold_prob2:0.7})
        print ("step {}, training accuracy {}".format(n, train_accuracy))
        
        test_accuracy = np.mean([sess.run(acc, feed_dict={x:testX, y_true:testY,hold_prob1:0.7,hold_prob2:0.8})for i in range(10)])


        print("test accuracy: {}".format(test_accuracy))


0
step 0, training accuracy 0.2199999988079071
test accuracy: 0.3314666748046875
1
step 1, training accuracy 0.25999999046325684
test accuracy: 0.3282666504383087
2
step 2, training accuracy 0.3400000035762787
test accuracy: 0.3333333134651184
3
step 3, training accuracy 0.4399999976158142
test accuracy: 0.3352000117301941
4
step 4, training accuracy 0.2800000011920929
test accuracy: 0.3330666422843933
5
step 5, training accuracy 0.3400000035762787
test accuracy: 0.33266669511795044
6
step 6, training accuracy 0.4000000059604645
test accuracy: 0.3274666666984558
7
step 7, training accuracy 0.3799999952316284
test accuracy: 0.3277333378791809
8
step 8, training accuracy 0.20000000298023224
test accuracy: 0.3258666694164276
9
step 9, training accuracy 0.2199999988079071
test accuracy: 0.33133330941200256
10
step 10, training accuracy 0.2800000011920929
test accuracy: 0.3293333649635315
11
step 11, training accuracy 0.3199999928474426
test accuracy: 0.3319999873638153
12
step 12, training

KeyboardInterrupt: 

In [31]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_input_fn, train_iterator_initializer_hook = get_inputs(trainX, trainY)
#     test_input_fn, test_iterator_initializer_hook = get_inputs(testX, testY)
    
    for i in range(30):

        sess.run(train, feed_dict={x: train_input_fn, y_true: train_iterator_initializer_hook,
                                        hold_prob1:0.5,hold_prob2:0.5})
        print ("step {}, training accuracy {}".format(i, train_accuracy))

#     X = mnist.test.images.reshape(10, 1000, 784)
#     Y = mnist.test.labels.reshape(10, 1000, 10)
    test_accuracy = np.mean([sess.run(acc, 
                             feed_dict={x:testX, y_true:testY,hold_prob1:1.0,hold_prob2:1.0})
                             for i in range(10)])

print ("test accuracy: {}".format(test_accuracy))


TypeError: float() argument must be a string or a number, not 'function'

In [ ]:
# 

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    
    train_batch, label_batch = tf.train.shuffle_batch([trainX, trainY], enqueue_many=True, batch_size=100, \
                                                      capacity=100, min_after_dequeue=10, allow_smaller_final_batch=True)

    for i in range(30):
# #         batch = mnist.train.next_batch(50)
#         idx = np.random.choice(np.arange(len(trainX)), 100, replace=False)
#         x_sample = trainX[idx]
#         print (x_sample[1])
#         y_sample = trainY[idx]
#         print (y_sample[1])

#         if i % 100 == 0:
        
        train_accuracy = sess.run(acc, feed_dict={x: train_batch, 
                                                           y_true: label_batch,
                                                           hold_prob1:0.5,hold_prob2:0.5})
        print ("step {}, training accuracy {}".format(i, train_accuracy))

        sess.run(train, feed_dict={x: train_batch, y_true: label_batch,
                                        hold_prob1:0.5,hold_prob2:0.5})
        print ("step {}, training accuracy {}".format(i, train_accuracy))

#     X = mnist.test.images.reshape(10, 1000, 784)
#     Y = mnist.test.labels.reshape(10, 1000, 10)
    test_accuracy = np.mean([sess.run(acc, 
                             feed_dict={x:testX, y_true:testY,hold_prob1:1.0,hold_prob2:1.0})
                             for i in range(10)])

print ("test accuracy: {}".format(test_accuracy))



In [ ]:
#Starting Empty lists to keep results
acc_list = []
auc_list = []
loss_list = []
saver = tf.train.Saver()

In [ ]:
#GPU settings
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.allocator_type = 'BFC'
with tf.Session(config=config) as sess:
    sess.run(init)
#     for i in range(epochs):
#         for j in range(0,steps-remaining,step_size):
#             #Feeding step_size-amount data with 0.5 keeping probabilities on DROPOUT LAYERS
#             _,c = sess.run([train,cross_entropy],  feed_dict={x:X[j:j+step_size] , y_true:Y[j:j+step_size],hold_prob1:0.5,hold_prob2:0.5})

    _,c = sess.run([train,cross_entropy],  feed_dict= { x: trainX , y_true:trainY,hold_prob1:0.5,hold_prob2:0.5})


    acc_on_cv,loss_on_cv,preds = sess.run([acc,cross_entropy,tf.nn.softmax(y_pred)],
        feed_dict={x:testX ,y_true:testY ,hold_prob1:1.0,hold_prob2:1.0})


In [ ]:
#GPU settings
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.allocator_type = 'BFC'
with tf.Session(config=config) as sess:
    sess.run(init)
# Train the model
#     c  = tf.estimator.inputs.numpy_input_fn(
#             x={"x": trainX},
#             y=trainY,
#             batch_size=100,
#             num_epochs=10,
#             shuffle=False)
    
    
    
#     eval_input_fn = tf.estimator.inputs.numpy_input_fn(
#                     x={"x": testX},
#                     y=testY,
#                     num_epochs=1,
#                     shuffle=False)

    image_batch, label_batch = tf.train.shuffle_batch(
                              [trainX, trainY],
                              batch_size=32,
                              num_threads=4,
                              capacity=50000,
                              min_after_dequeue=10000)


In [ ]:
image_batch

In [ ]:
dx = tf.data.Dataset.from_tensor_slices(trainX)
dy = tf.data.Dataset.from_tensor_slices(trainY)
dcomb = tf.data.Dataset.zip((dx, dy)).batch(3)
iterator = dcomb.make_initializable_iterator()
    # extract an element
next_element = iterator.get_next()
with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(15):
        val = sess.run(next_element)
        print(val)
        if (i + 1) % (10 // 3) == 0 and i > 0:
            sess.run(iterator.initializer)
            
            _,c = sess.run([train,cross_entropy],  feed_dict= { x: trainX , y_true:trainY,hold_prob1:0.5,hold_prob2:0.5})


In [ ]:
# create the training datasets
dx_train = tf.data.Dataset.from_tensor_slices(trainX)
# apply a one-hot transformation to each label for use in the neural network
dy_train = tf.data.Dataset.from_tensor_slices(trainY)
# zip the x and y training data together and shuffle, batch etc.
train_dataset = tf.data.Dataset.zip((dx_train, dy_train)).shuffle(500).repeat().batch(30)

In [ ]:
# do the same operations for the validation set
dx_valid = tf.data.Dataset.from_tensor_slices(testX)
dy_valid = tf.data.Dataset.from_tensor_slices(testY)
valid_dataset = tf.data.Dataset.zip((dx_valid, dy_valid)).shuffle(500).repeat().batch(30)

In [ ]:
# create general iterator
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
next_element = iterator.get_next()

In [ ]:
#  make datasets that we can initialize separately, but using the same structure via the common iterator
training_init_op = iterator.make_initializer(train_dataset)
validation_init_op = iterator.make_initializer(valid_dataset)

In [ ]:
init_op = tf.global_variables_initializer()

In [ ]:
# run the training
epochs = 600
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(training_init_op)
    for i in range(epochs):
        l, _, acc_1 = sess.run([cross_entropy, train, acc])
        if i % 50 == 0:
            print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, l, acc * 100))
    # now setup the validation run
    valid_iters = 100
    # re-initialize the iterator, but this time with validation data
    sess.run(validation_init_op)
    avg_acc = 0
    for i in range(valid_iters):
        acc_2 = sess.run([acc])
        avg_acc += acc_2[0]
    print("Average validation set accuracy over {} iterations is {:.2f}%".format(valid_iters,(avg_acc / valid_iters) * 100))

In [ ]:
type(trainX)

In [ ]:
idx = np.random.choice(np.arange(len(trainX)), 1000, replace=False)
x_sample = trainX[idx]
y_sample = trainY[idx]

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(30):
#         batch = mnist.train.next_batch(50)
        idx = np.random.choice(np.arange(len(trainX)), 1000, replace=False)
        x_sample = trainX[idx]
        y_sample = trainY[idx]

        if i % 100 == 0:
            train_accuracy = sess.run(acc, feed_dict={x: x_sample, 
                                                           y_: y_sample,
                                                           keep_prob: 0.7})
            print "step {}, training accuracy {}".format(i, train_accuracy)

        sess.run(train, feed_dict={x: x_sample, y_: y_sample,
                                        keep_prob: 0.5})

#     X = mnist.test.images.reshape(10, 1000, 784)
#     Y = mnist.test.labels.reshape(10, 1000, 10)
    test_accuracy = np.mean([sess.run(acc, 
                             feed_dict={x:test_, y_:Y[i],keep_prob:1.0}) 
                             for i in range(10)])

print "test accuracy: {}".format(test_accuracy)

In [ ]:
acc_on_cv

In [ ]:
##############################################################################################################################

In [ ]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))
 
def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

In [ ]:

def create_convolutional_layer(input,
               num_input_channels, 
               conv_filter_size,        
               num_filters):  
    
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)

    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    layer += biases

    ## We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    ## Output of pooling is fed to Relu which is the activation function for us.
    layer = tf.nn.relu(layer)

    return layer

In [ ]:
def create_flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])

    return layer

In [ ]:

def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    #Let's define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)

    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [ ]:

x = tf.placeholder(tf.float32, shape=[None, 32,32,3], name='x')

y_true = tf.placeholder(tf.float32, shape=[None, 3], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [ ]:
num_channels

In [ ]:

layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=num_channels,
               conv_filter_size=filter_size_conv1,
               num_filters=num_filters_conv1)

layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters_conv1,
               conv_filter_size=filter_size_conv2,
               num_filters=num_filters_conv2)

layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=num_filters_conv2,
               conv_filter_size=filter_size_conv3,
               num_filters=num_filters_conv3)
          
layer_flat = create_flatten_layer(layer_conv3)

layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=fc_layer_size,
                     use_relu=True)

layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_layer_size,
                     num_outputs=num_classes,
                     use_relu=False)